In [55]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
import seaborn as sns
sns.set_style('darkgrid')
import plotly.graph_objects as go

In [56]:
# -------------------
# Load Training Data
# -------------------
dataset_train = pd.read_csv('/content/TCS Stock 2013_2020_Train.csv')
print(dataset_train.columns)   # check column names

training_set = dataset_train[['Open ']].values   # ✅ use correct column name

Index(['Date ', 'Open ', 'High ', 'Low ', 'Close ', 'Shares Traded '], dtype='object')


In [57]:
dataset_train

,Date,Open,High,Low,Close,Shares Traded
0,1-Jan-13,5937.65,5963.90,5935.20,5950.85,77902745.0
1,2-Jan-13,5982.60,6006.05,5982.00,5993.25,116057389.0
2,3-Jan-13,6015.80,6017.00,5986.55,6009.50,99989933.0
3,4-Jan-13,6011.95,6020.75,5981.55,6016.15,113232990.0
4,7-Jan-13,6042.15,6042.15,5977.15,5988.40,110248018.0
...,...,...,...,...,...,...
1975,24-Dec-20,13672.15,13771.75,13626.90,13749.25,471367730.0
1976,28-Dec-20,13815.15,13885.30,13811.55,13873.20,403634030.0
1977,29-Dec-20,13910.35,13967.60,13859.90,13932.60,439593961.0
1978,30-Dec-20,13980.90,13997.00,13864.95,13981.95,380681073.0


In [58]:
# Scale data
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [59]:
# Create X_train, y_train
X_train, y_train = [], []
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [60]:
# -------------------
# Build LSTM Model
# -------------------
model = Sequential()
model.add(Input(shape=(60, 1)))     # ✅ Correct input shape
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                   │ (None, 60, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 60, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,985 (195.25 KB)

 Trainable params: 49,985 (195.25 KB)

 Non-trainable params: 0 (0.00 B)

In [61]:
# Train
model.fit(X_train, y_train, epochs=200, batch_size=32)

Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0511
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0030
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0028
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0028
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0023
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0026
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0022
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0021
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0021
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0019
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0018
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0020
Epoch 13/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0016
Epoch 14/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0017
Epoch 15/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0016
Ep

In [62]:
# -------------------
# Test Data
# -------------------
dataset_test = pd.read_csv('/content/TCS Stock 2021_2024_Test.csv')
real_stock_price = dataset_test[['Open ']].values   # ✅ ensure correct col

In [63]:
dataset_test

,Date,Open,High,Low,Close,Shares Traded
0,1-Jan-21,13996.10,14049.85,13991.35,14018.50,258090905.0
1,4-Jan-21,14104.35,14147.95,13953.75,14132.90,494999295.0
2,5-Jan-21,14075.15,14215.60,14048.15,14199.50,492475349.0
3,6-Jan-21,14240.95,14244.15,14039.90,14146.25,632323316.0
4,7-Jan-21,14253.75,14256.25,14123.10,14137.35,559173512.0
...,...,...,...,...,...,...
986,24-Dec-24,23769.10,23867.65,23685.15,23727.65,177664746.0
987,26-Dec-24,23775.80,23854.50,23653.60,23750.20,177681783.0
988,27-Dec-24,23801.40,23938.85,23800.60,23813.40,176821521.0
989,30-Dec-24,23796.90,23915.35,23599.30,23644.90,364929453.0


In [64]:
# Combine train + test for inputs
dataset_total = pd.concat((dataset_train['Open '], dataset_test['Open ']), axis=0)

inputs = dataset_total[len(dataset_total)-len(dataset_test)-60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [65]:
# Create X_test
X_test = []
for i in range(60, len(inputs)):
    X_test.append(inputs[i-60:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [66]:
# Predict
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [67]:
# Get the dates from the test dataset
dates = dataset_test['Date ']

# Create traces for real and predicted stock prices
real_price_trace = go.Scatter(
    x=dates,
    y=real_stock_price.flatten(),
    mode='lines',
    name='Real Stock Price',
    line=dict(color='red')
)

predicted_price_trace = go.Scatter(
    x=dates,
    y=predicted_stock_price.flatten(),
    mode='lines',
    name='Predicted Stock Price',
    line=dict(color='blue')
)

# Create layout
layout = go.Layout(
    title='TCS Stock Price Prediction (LSTM) with Year on Hover',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Stock Price'),
    hovermode='x unified' # This enables showing data from both traces on hover
)

# Create figure and add traces
fig = go.Figure(data=[real_price_trace, predicted_price_trace], layout=layout)

# Display the plot
fig.show()